In [10]:
!pip install geemap
!pip install geehydro
!pip install ipygee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10143 sha256=24e156f82ccf63109d78299cc1854ccfdcfbaaf4d66d76e1e12719648eb2c2a5
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 13.3 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
  Created wheel for ipygee: filename=ipygee-0.0.17-py3-none-any.whl size=40379 sha256=5b68c08b7be62a17f3c0f03438eec13df61597bec4fc2c7ef5b26a10594235a4
  Stored in directory: /root/.cache/pip/wheels/73/af/7e/a9d3fac7d32e649c63fdbdb9b320181b20eb4bfdc80af13974
  Created wheel for geetools: filename

In [11]:
import ee
import geemap
import matplotlib.pyplot as plt 
import folium 
import geehydro
import numpy as np # yes, numpy!
import pandas as pd # yes, pandas!
import matplotlib.pyplot as plt 
from ipygee import*
from pandas.plotting import register_matplotlib_converters



In [2]:
Map = geemap.Map()
Map


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=acnOHyUHSsNjy67Suf9BCVKi4oJsS9aga0cW01u6Ksc&tc=8IWzpB5En4Dr0xi8EG2Yi8NsJRM27RqyVKQ7SxUZBg8&cc=alKEz7L9StAeDwjhARgFBQOen9GsJA0Xrqu70XXD558

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qhPq67UanmpTwtVThkp3IGazk09xWLM5wh2y3ayLVgz_4XG84Swcy4

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
Map.draw_features

In [7]:
roi = Map.draw_last_feature
Map.addLayer(roi)

In [14]:
print(roi.geometry().getInfo())


{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[20.006104, -23.333429], [19.995117, -24.807928], [20.665283, -25.444515], [20.852051, -26.107354], [20.577393, -26.530794], [20.698242, -26.874306], [20.906982, -26.79588], [21.368408, -26.825296], [21.75293, -26.825296], [21.785889, -26.658505], [22.038574, -26.629046], [22.456055, -26.196109], [22.598877, -26.186251], [22.895508, -25.444515], [23.203125, -25.26581], [23.455811, -25.285679], [23.917236, -25.613048], [24.180908, -25.63286], [24.532471, -25.761557], [24.499512, -25.63286], [24.246826, -25.513939], [24.246826, -24.987303], [24.093018, -24.847812], [23.071289, -24.827871], [23.049316, -23.313252], [20.006104, -23.333429]]]}


In [8]:
# Select modis image collection to get NDVI
modis = ee.ImageCollection('MODIS/006/MOD13Q1')

min_date = '2000-01-01'
max_date = '2020-12-01'

modis = modis.filterDate(ee.DateRange(min_date, max_date))

available_dates = modis.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
).getInfo()
print("available dates, from {} to {}".format(available_dates[0], available_dates[-1]))

# select EVI and NDVI
evi = modis.select('EVI')
ndvi = modis.select('NDVI')
def scale_factor(image):
  # scale factor for the MODIS MOD13Q1 product
  return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)
Map.addLayer(scaled_ndvi.mean().clip(roi),
                  vis_params={'min': -1,
                              'max': 1,
                              'palette': ['red', 'yellow','green']})
Map.centerObject(roi)

Map

available dates, from 2000-02-18 to 2020-11-16


Map(bottom=18934.0, center=[-23.986252599841798, 18.984375000000004], controls=(WidgetControl(options=['positi…

In [12]:
selected_roi_ndvi = chart.Image.series(**{'imageCollection': scaled_ndvi,
                                   'region': roi,
                                   'reducer': ee.Reducer.mean(),
                                   'scale': 1000,
                                   'xProperty': 'system:time_start'})
selected_roi_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [13]:
# Timeseries data can be extracted as a Panda's dataframe 
Kgalagadi_ndvi = selected_roi_ndvi.dataframe
Kgalagadi_ndvi

,NDVI
2000-02-18,0.331070
2000-03-05,0.322955
2000-03-21,0.330260
2000-04-06,0.312373
2000-04-22,0.288323
...,...
2020-09-13,0.194875
2020-09-29,0.190158
2020-10-15,0.199189
2020-10-31,0.204482
